In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

# Load data
train_df = pd.read_csv('train_data_movie_rate.csv')
test_df = pd.read_csv('test_data_q3.csv')

# Prepare data for Surprise (SVD)
reader = Reader(rating_scale=(0, 5))  # Based on your rating scale
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'label']], reader)

# Split data for SVD
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD model
svd_model = SVD(n_factors=150, lr_all=0.005, reg_all=0.08, random_state=42, n_epochs=50)
svd_model.fit(trainset)

# Evaluate SVD
svd_predictions = svd_model.test(testset)
print("SVD RMSE:", accuracy.rmse(svd_predictions))
print("SVD MAE:", accuracy.mae(svd_predictions))

# Prepare data for Random Forest
# Create user and item features
user_stats = train_df.groupby('user_id')['label'].agg(['mean', 'count', 'std']).add_prefix('user_')
item_stats = train_df.groupby('item_id')['label'].agg(['mean', 'count', 'std']).add_prefix('item_')

# Merge features
train_features = train_df.merge(user_stats, on='user_id').merge(item_stats, on='item_id')
test_features = test_df.merge(user_stats, on='user_id', how='left').merge(item_stats, on='item_id', how='left')

# Fill NA values for cold-start cases
train_features.fillna(train_features.mean(), inplace=True)
test_features.fillna(train_features.mean(), inplace=True)

# Select features
feature_cols = ['user_mean', 'user_count', 'user_std', 'item_mean', 'item_count', 'item_std']
X_train = train_features[feature_cols]
y_train = train_features['label']
X_test = test_features[feature_cols]

# Train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Generate predictions
svd_preds = []
for _, row in test_df.iterrows():
    uid = row['user_id']
    iid = row['item_id']
    pred = svd_model.predict(uid, iid).est
    svd_preds.append(pred)

rf_preds = rf_model.predict(X_test)

# Create ensemble predictions (80% SVD, 20% RF)
submission = pd.DataFrame({
    'id': test_df['id'],
    'label': 0.8*np.array(svd_preds) + 0.2*np.array(rf_preds)
})

# Ensure ratings are within valid range and rounded to nearest 0.5
submission['label'] = np.clip(submission['label'], 0.5, 4.0)
submission['label'] = np.round(submission['label'] * 2) / 2  # Round to nearest 0.5

# Verify distribution
print("\nPrediction distribution:")
print(submission['label'].value_counts().sort_index())

# Save submission
submission.to_csv('submission3.csv', index=False)
print("\nSubmission file saved as 'submission.csv'")

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import optuna

# Load data
train_df = pd.read_csv('train_data_movie_rate.csv')
test_df = pd.read_csv('test_data_q3.csv')

# Prepare data for Surprise (SVD)
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'label']], reader)

# Split data for SVD
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Feature Engineering
user_stats = train_df.groupby('user_id')['label'].agg(['mean', 'count', 'std']).add_prefix('user_')
item_stats = train_df.groupby('item_id')['label'].agg(['mean', 'count', 'std']).add_prefix('item_')
train_features = train_df.merge(user_stats, on='user_id').merge(item_stats, on='item_id')
test_features = test_df.merge(user_stats, on='user_id', how='left').merge(item_stats, on='item_id', how='left')

# Fill NA values
train_features = train_features.fillna(train_features.mean())
test_features = test_features.fillna(train_features.mean())

# Select features for Random Forest
feature_cols = ['user_mean', 'user_count', 'user_std', 'item_mean', 'item_count', 'item_std']
X_train = train_features[feature_cols]
y_train = train_features['label']
X_test = test_features[feature_cols]

# Get items present in the trainset for validation
train_items = set(trainset.all_items())  # Internal item IDs in trainset
train_raw_items = set(trainset.to_raw_iid(iid) for iid in train_items)  # Convert to raw item IDs

# Filter testset to only include items present in trainset for validation
filtered_testset = [(uid, iid, r) for uid, iid, r in testset if iid in train_raw_items]

# Define objective function for Optuna
def objective(trial):
    # SVD hyperparameters
    svd_params = {
        'n_factors': trial.suggest_int('n_factors', 50, 200),
        'lr_all': trial.suggest_float('lr_all', 0.001, 0.01, log=True),
        'reg_all': trial.suggest_float('reg_all', 0.02, 0.2, log=True),
        'n_epochs': trial.suggest_int('n_epochs', 20, 50),
        'random_state': 42
    }

    # Random Forest hyperparameters
    rf_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 10, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'random_state': 42
    }

    # Train SVD model
    svd_model = SVD(**svd_params)
    svd_model.fit(trainset)

    # Train Random Forest
    rf_model = RandomForestRegressor(**rf_params)
    rf_model.fit(X_train, y_train)

    # Evaluate ensemble on filtered validation set
    svd_preds = []
    valid_indices = []
    for idx, (uid, iid, _) in enumerate(filtered_testset):
        pred = svd_model.predict(uid, iid).est
        svd_preds.append(pred)
        # Find the corresponding index in train_features for this user-item pair
        train_idx = train_df[(train_df['user_id'] == uid) & (train_df['item_id'] == iid)].index
        if not train_idx.empty:
            valid_indices.append(train_idx[0])

    # Ensure we have valid indices for Random Forest predictions
    if not valid_indices:
        return float('inf')  # Skip trial if no valid indices

    rf_preds = rf_model.predict(X_train.loc[valid_indices])
    ensemble_preds = 0.8 * np.array(svd_preds) + 0.2 * np.array(rf_preds)
    true_ratings = np.array([r for _, _, r in filtered_testset])
    rmse = np.sqrt(np.mean((ensemble_preds - true_ratings) ** 2))

    return rmse

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Best parameters
print("Best SVD and RF parameters:", study.best_params)
print("Best RMSE:", study.best_value)

# Train final models with best parameters
best_svd_params = {
    'n_factors': study.best_params['n_factors'],
    'lr_all': study.best_params['lr_all'],
    'reg_all': study.best_params['reg_all'],
    'n_epochs': study.best_params['n_epochs'],
    'random_state': 42
}

best_rf_params = {
    'n_estimators': study.best_params['n_estimators'],
    'max_depth': study.best_params['max_depth'],
    'min_samples_split': study.best_params['min_samples_split'],
    'min_samples_leaf': study.best_params['min_samples_leaf'],
    'random_state': 42
}

# Train final SVD model
svd_model = SVD(**best_svd_params)
svd_model.fit(trainset)

# Train final Random Forest
rf_model = RandomForestRegressor(**best_rf_params)
rf_model.fit(X_train, y_train)

# Evaluate on validation set
svd_predictions = svd_model.test(testset)
print("Final SVD RMSE:", accuracy.rmse(svd_predictions))
print("Final SVD MAE:", accuracy.mae(svd_predictions))

# Generate predictions for test set (handle cold-start items)
svd_preds = []
for _, row in test_df.iterrows():
    uid = row['user_id']
    iid = row['item_id']
    try:
        pred = svd_model.predict(uid, iid).est
    except ValueError:
        # Fallback for cold-start items: use item average or global mean
        item_avg = item_stats.loc[item_stats.index == iid, 'item_mean'].iloc[0] if iid in item_stats.index else train_df['label'].mean()
        pred = item_avg
    svd_preds.append(pred)

rf_preds = rf_model.predict(X_test)

# Create ensemble predictions (80% SVD, 20% RF)
submission = pd.DataFrame({
    'id': test_df['id'],
    'label': 0.8 * np.array(svd_preds) + 0.2 * np.array(rf_preds)
})

# Ensure ratings are within valid range and rounded to nearest 0.5
submission['label'] = np.clip(submission['label'], 0.5, 4.0)
submission['label'] = np.round(submission['label'] * 2) / 2

# Verify distribution
print("\nPrediction distribution:")
print(submission['label'].value_counts().sort_index())

# Save submission
submission.to_csv('submission4.csv', index=False)
print("\nSubmission file saved as 'submission3.csv'")

/mnt/c/Users/Lenovo/Desktop/DS/DSCA3/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-14 05:33:53,611] A new study created in memory with name: no-name-d889d526-f8c0-42b6-8710-98276bdea41e
[I 2025-05-14 05:34:34,368] Trial 0 finished with value: 0.7163855049975403 and parameters: {'n_factors': 109, 'lr_all': 0.0019404384530604544, 'reg_all': 0.18254933912911264, 'n_epochs': 32, 'n_estimators': 174, 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7163855049975403.
[I 2025-05-14 05:34:58,004] Trial 1 finished with value: 0.7355563511373718 and parameters: {'n_factors': 164, 'lr_all': 0.004030068968992297, 'reg_all': 0.0648060426077171, 'n_epochs': 35, 'n_estimators': 115, 'max_depth': 49, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is 

Best SVD and RF parameters: {'n_factors': 81, 'lr_all': 0.004263579201729637, 'reg_all': 0.047678769035501, 'n_epochs': 38, 'n_estimators': 109, 'max_depth': 46, 'min_samples_split': 4, 'min_samples_leaf': 1}
Best RMSE: 0.7043272339980249
RMSE: 0.7988
Final SVD RMSE: 0.7988096035600628
MAE:  0.6216
Final SVD MAE: 0.6216312451526894

Prediction distribution:
label
1.5      2
2.0     57
2.5    384
3.0    899
3.5    357
4.0     17
Name: count, dtype: int64

Submission file saved as 'submission3.csv'


In [2]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

# Load data
train_df = pd.read_csv('train_data_movie_rate.csv')
test_df = pd.read_csv('test_data_q3.csv')

# Load trust data if available
try:
    trust_df = pd.read_csv('train_data_movie_trust.csv')
    trust_features = trust_df.groupby('user_id_trustee')['trust_value'].mean().reset_index(name='avg_trust')
except FileNotFoundError:
    trust_features = pd.DataFrame(columns=['user_id_trustee', 'avg_trust'])
    print("Warning: train_data_movie_trust.csv not found. Proceeding without trust features.")

# Prepare data for Surprise (SVD)
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'label']], reader)

# Split data for initial validation
trainset, val_set = train_test_split(data, test_size=0.2, random_state=42)

# Feature Engineering
user_stats = train_df.groupby('user_id')['label'].agg(['mean', 'count', 'std']).add_prefix('user_')
item_stats = train_df.groupby('item_id')['label'].agg(['mean', 'count', 'std']).add_prefix('item_')
train_features = train_df.merge(user_stats, on='user_id').merge(item_stats, on='item_id')
test_features = test_df.merge(user_stats, on='user_id', how='left').merge(item_stats, on='item_id', how='left')

# Merge trust features if available
if not trust_features.empty:
    train_features = train_features.merge(trust_features, left_on='user_id', right_on='user_id_trustee', how='left')
    test_features = test_features.merge(trust_features, left_on='user_id', right_on='user_id_trustee', how='left')
    train_features['avg_trust'] = train_features['avg_trust'].fillna(0)
    test_features['avg_trust'] = test_features['avg_trust'].fillna(0)

# Fill NA values
train_features = train_features.fillna(train_features.mean())
test_features = test_features.fillna(train_features.mean())

# Select features for Random Forest
feature_cols = ['user_mean', 'user_count', 'user_std', 'item_mean', 'item_count', 'item_std']
if not trust_features.empty:
    feature_cols.append('avg_trust')
X_train = train_features[feature_cols]
y_train = train_features['label']
X_test = test_features[feature_cols]

# Scale features for Random Forest
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# GridSearchCV for SVD
svd_param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.08, 0.1],
    'n_epochs': [20, 30, 40]
}
gs = GridSearchCV(SVD, svd_param_grid, measures=['rmse'], cv=3, n_jobs=-1)
gs.fit(data)

# Best SVD parameters and model
print("Best SVD RMSE:", gs.best_score['rmse'])
print("Best SVD parameters:", gs.best_params['rmse'])
best_svd_model = gs.best_estimator['rmse']
best_svd_model.fit(trainset)

# Evaluate SVD on validation set
svd_predictions = best_svd_model.test(val_set)
print("SVD RMSE on validation set:", accuracy.rmse(svd_predictions))
print("SVD MAE on validation set:", accuracy.mae(svd_predictions))

# Train Random Forest with cross-validation for robustness
rf_model = RandomForestRegressor(
    n_estimators=150,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf_cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Random Forest CV RMSE (mean):", np.sqrt(-rf_cv_scores.mean()))
rf_model.fit(X_train_scaled, y_train)

# Optimize ensemble weights
svd_val_preds = np.array([pred.est for pred in best_svd_model.test(val_set)])
rf_val_preds = rf_model.predict(X_train_scaled[:len(val_set)])
weights = np.linspace(0.6, 0.9, 7)  # Test weights from 60% to 90% for SVD
best_rmse = float('inf')
best_weight = 0.8

for w in weights:
    ensemble_preds = w * svd_val_preds + (1 - w) * rf_val_preds
    rmse = np.sqrt(np.mean((ensemble_preds - np.array([r for _, _, r in val_set])) ** 2))
    if rmse < best_rmse:
        best_rmse = rmse
        best_weight = w

print(f"Best ensemble weight for SVD: {best_weight:.2f}, RMSE: {best_rmse:.4f}")

# Generate predictions for test set
svd_test_preds = np.array([best_svd_model.predict(uid, iid).est for uid, iid in zip(test_df['user_id'], test_df['item_id'])])
rf_test_preds = rf_model.predict(X_test_scaled)

# Create ensemble predictions with optimized weight
submission = pd.DataFrame({
    'id': test_df['id'],
    'label': best_weight * svd_test_preds + (1 - best_weight) * rf_test_preds
})

# Ensure ratings are within valid range and rounded to nearest 0.5
submission['label'] = np.clip(submission['label'], 0.5, 4.0)
submission['label'] = np.round(submission['label'] * 2) / 2

# Verify distribution
print("\nPrediction distribution:")
print(submission['label'].value_counts().sort_index())

# Save submission
submission.to_csv('submission4.csv', index=False)
print("\nSubmission file saved as 'submission4.csv'")

Best SVD RMSE: 0.7919467728715471
Best SVD parameters: {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.08, 'n_epochs': 40}
RMSE: 0.7890
SVD RMSE on validation set: 0.7889835967890491
MAE:  0.6099
SVD MAE on validation set: 0.6099137871398527
Random Forest CV RMSE (mean): 0.7495263364373492
Best ensemble weight for SVD: 0.90, RMSE: 0.7919

Prediction distribution:
label
1.5      2
2.0     60
2.5    386
3.0    907
3.5    339
4.0     22
Name: count, dtype: int64

Submission file saved as 'submission4.csv'


In [8]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV as SurpriseGridSearchCV
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV as SklearnGridSearchCV
from sklearn.model_selection import cross_val_score

# Load data
train_df = pd.read_csv('train_data_movie_rate.csv')
test_df = pd.read_csv('test_data_q3.csv')

# Load trust data if available
try:
    trust_df = pd.read_csv('train_data_movie_trust.csv')
    trust_features = trust_df.groupby('user_id_trustee')['trust_value'].mean().reset_index(name='avg_trust')
except FileNotFoundError:
    trust_features = pd.DataFrame(columns=['user_id_trustee', 'avg_trust'])
    print("Warning: train_data_movie_trust.csv not found. Proceeding without trust features.")

# Prepare data for Surprise (SVD)
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'label']], reader)

# Split data for SVD
trainset, val_set = train_test_split(data, test_size=0.2, random_state=42)

# Feature Engineering
user_stats = train_df.groupby('user_id')['label'].agg(['mean', 'count', 'std']).add_prefix('user_')
item_stats = train_df.groupby('item_id')['label'].agg(['mean', 'count', 'std']).add_prefix('item_')
train_features = train_df.merge(user_stats, on='user_id').merge(item_stats, on='item_id')
test_features = test_df.merge(user_stats, on='user_id', how='left').merge(item_stats, on='item_id', how='left')

# Merge trust features if available
if not trust_features.empty:
    train_features = train_features.merge(trust_features, left_on='user_id', right_on='user_id_trustee', how='left')
    test_features = test_features.merge(trust_features, left_on='user_id', right_on='user_id_trustee', how='left')
    train_features['avg_trust'] = train_features['avg_trust'].fillna(0)
    test_features['avg_trust'] = test_features['avg_trust'].fillna(0)

# Add interaction feature
train_features['user_item_diff'] = train_features['user_mean'] - train_features['item_mean']
test_features['user_item_diff'] = test_features['user_mean'] - test_features['item_mean']

# Fill NA values
train_features = train_features.fillna(train_features.mean())
test_features = test_features.fillna(test_features.mean())

# Select features
feature_cols = ['user_mean', 'user_count', 'user_std', 'item_mean', 'item_count', 'item_std']
if not trust_features.empty:
    feature_cols.append('avg_trust')
feature_cols.append('user_item_diff')
X_train = train_features[feature_cols]
y_train = train_features['label']
X_test = test_features[feature_cols]

# Scale features for XGBoost
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# GridSearchCV for SVD
svd_param_grid = {
    'n_factors': [100, 150, 200],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.05, 0.08, 0.1],
    'n_epochs': [30, 40, 50]
}
gs_svd = SurpriseGridSearchCV(SVD, svd_param_grid, measures=['rmse'], cv=3, n_jobs=-1)
gs_svd.fit(data)

# Best SVD parameters and model
print("Best SVD RMSE:", gs_svd.best_score['rmse'])
print("Best SVD parameters:", gs_svd.best_params['rmse'])
best_svd_model = gs_svd.best_estimator['rmse']
best_svd_model.fit(trainset)

# Evaluate SVD on validation set
svd_val_preds = np.array([pred.est for pred in best_svd_model.test(val_set)])
true_val_ratings = np.array([r for _, _, r in val_set])
svd_rmse = np.sqrt(np.mean((svd_val_preds - true_val_ratings) ** 2))
print("SVD RMSE on validation set:", svd_rmse)

# GridSearchCV for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
    'reg_alpha': [0, 0.1],  # L1 regularization
    'reg_lambda': [1, 1.5],  # L2 regularization
    'random_state': [42]
}
xgb_model = XGBRegressor()
gs_xgb = SklearnGridSearchCV(xgb_model, xgb_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gs_xgb.fit(X_train_scaled, y_train)

# Best XGBoost parameters and model
print("Best XGBoost RMSE:", np.sqrt(-gs_xgb.best_score_))
print("Best XGBoost parameters:", gs_xgb.best_params_)
best_xgb_model = gs_xgb.best_estimator_

# Evaluate XGBoost with cross-validation
xgb_cv_scores = cross_val_score(best_xgb_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
xgb_rmse_mean = np.sqrt(-xgb_cv_scores.mean())
print("XGBoost CV RMSE (mean):", xgb_rmse_mean)
best_xgb_model.fit(X_train_scaled, y_train)

# Optimize ensemble weights
xgb_val_preds = best_xgb_model.predict(X_train_scaled[:len(val_set)])
weights = np.linspace(0.6, 0.9, 7)  # Test weights from 60% to 90% for SVD
best_rmse = float('inf')
best_weight = 0.8

for w in weights:
    ensemble_preds = w * svd_val_preds + (1 - w) * xgb_val_preds
    rmse = np.sqrt(np.mean((ensemble_preds - true_val_ratings) ** 2))
    if rmse < best_rmse:
        best_rmse = rmse
        best_weight = w

print(f"Best ensemble weight for SVD: {best_weight:.2f}, RMSE: {best_rmse:.4f}")

# Generate predictions for test set
svd_test_preds = np.array([best_svd_model.predict(uid, iid).est for uid, iid in zip(test_df['user_id'], test_df['item_id'])])
xgb_test_preds = best_xgb_model.predict(X_test_scaled)

# Create ensemble predictions with optimized weight
submission = pd.DataFrame({
    'id': test_df['id'],
    'label': best_weight * svd_test_preds + (1 - best_weight) * xgb_test_preds
})

# Handle cold-start with confidence weighting
global_mean = train_df['label'].mean()
submission['label'] = np.where(
    test_features['item_count'].isna(),
    global_mean,
    submission['label']
)

# Ensure ratings are within valid range and rounded to nearest 0.5
submission['label'] = np.clip(submission['label'], 0.5, 4.0)
submission['label'] = np.round(submission['label'] * 2) / 2

# Verify distribution
print("\nPrediction distribution:")
print(submission['label'].value_counts().sort_index())

# Save submission
submission.to_csv('submission5.csv', index=False)
print("\nSubmission file saved as 'submission5.csv'")

Best SVD RMSE: 0.7921246785369758
Best SVD parameters: {'n_factors': 150, 'lr_all': 0.01, 'reg_all': 0.1, 'n_epochs': 50}
SVD RMSE on validation set: 0.7883861821461952
Best XGBoost RMSE: 0.7272786708348726
Best XGBoost parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150, 'random_state': 42, 'reg_alpha': 0.1, 'reg_lambda': 1.5, 'subsample': 0.9}
XGBoost CV RMSE (mean): 0.7272786708348726
Best ensemble weight for SVD: 0.90, RMSE: 0.7913

Prediction distribution:
label
1.5      5
2.0     86
2.5    358
3.0    897
3.5    350
4.0     20
Name: count, dtype: int64

Submission file saved as 'submission5.csv'
